In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from crazyflie_description_py.parameters import MIC_POSITIONS, FS, N_BUFFER
import pyroomacoustics as pra
from audio_stack.beam_former import rotate_mics

def generate_rir(distance_cm=0, yaw_deg=0, single_mic=False, ax=None):
    # simplification1: attenuation normally depends on pressure, humidity, frequency.
    # simplification 2: no extra loss at wall
    a = 10 # attenuation coefficient in dB / m, between 100-200 degrees, 4000+ Hz
    c = 343 # m/s
    Ts = 1 / FS
    yaw_offset = -120
    y_offset = 0.1
    offset = [0, y_offset + distance_cm * 1e-2]
    delta = 0.1 # difference between mic and source
    
    if single_mic:
        mic_positions = np.array([offset[0] + delta, offset[1]]).reshape((1, len(offset)))
    else:
        mic_positions = np.array(MIC_POSITIONS)
        mic_positions = offset + rotate_mics(mic_positions, yaw_offset - yaw_deg)
            
    source = offset
    source_image = [0, -offset[1]]

    if ax is not None:
        for i, mic in enumerate(mic_positions):
            ax.scatter(*mic, label=f'mic{i}')
        ax.plot([-1, 1], [0, 0], label='wall')
        ax.axis('equal')
        ax.set_xlim(-0.5, 0.5)
        ax.set_ylim(-1.0, 1.0)
        ax.scatter(*source, label='buzzer')
        ax.scatter(*source_image, label='buzzer image')
        ax.legend()

    Hs = []
    for mic in mic_positions:
        direct_path = np.linalg.norm(mic - source)
        reflect_path = np.linalg.norm(mic - source_image)
        alpha0 = 10**(-a*direct_path/20)
        alpha1 = 10**(-a*reflect_path/20)
        n0 = direct_path / c 
        n1 = reflect_path / c
        
        H_ij = alpha0 * np.exp(-1j*2*np.pi*frequencies_theo*n0) + alpha1 * np.exp(-1j*2*np.pi*frequencies_theo*n1)
        Hs.append(H_ij)
    return Hs

In [ ]:
def generate_room(distance_cm=0, yaw_deg=0, ax=None, single_mic=False, source='mono4125'):
    room_dim = np.array([50, 20])
    yaw_offset = -120
    x_offset = room_dim[0] / 2
    y_offset = 0.1
    delta = 0.1 # difference between mic and source
    
    m = pra.Material(energy_absorption="glass_3mm")
    room = pra.ShoeBox(fs=FS, p=room_dim, max_order=1, materials=m)

    offset = [x_offset, y_offset + distance_cm * 1e-2]
    
    if single_mic:
        mic_positions = np.array([offset[0] + delta, offset[1]]).reshape((1, len(offset)))
    else:
        mic_positions = np.array(MIC_POSITIONS)
        mic_positions = offset + rotate_mics(mic_positions, yaw_offset - yaw_deg)
        
    signal = signals[key]
            
    room.add_source(offset, signal=signal)

    beam_former = pra.Beamformer(mic_positions.T,  room.fs)
    room.add_microphone_array(beam_former)
    
    if ax is not None:
        for i, mic in enumerate(mic_positions):
            ax.scatter(*mic, label=f'mic{i}')
        ax.plot([0, room_dim[0]], [0, 0], label='wall')
        ax.axis('equal')
        ax.set_xlim(x_offset - 0.5, x_offset + 0.5)
        ax.set_ylim(-0.1, 1.0)
        ax.scatter(*offset, label='buzzer')
        ax.legend()
    return room


def get_signals_f(room, n_buffer=N_BUFFER):
    room.simulate()
    signals_f = np.array(
        [
            pra.transform.stft.analysis(signal, N_BUFFER, N_BUFFER // 2).T
            for signal in room.mic_array.signals
        ]# n_mics x n_frequencies x n_times
    ).transpose(2, 0, 1)  # n_times, n_mics, n_frequencies
    return signals_f[:-10, ...]

def get_frequencies(fs, n_buffer=N_BUFFER):
    return np.arange(N_BUFFER // 2 + 1) * fs / N_BUFFER

In [ ]:
def matrix_slices(df_matrix, frequencies, distances):
    from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider, IntSlider
    import ipywidgets as widgets

    def update_figure(slice_f, slice_d):
        fig, axs = plt.subplots(1, 2)
        fig.set_size_inches(10, 5)
        axs[0].semilogy(distances, df_matrix[slice_f, :])
        axs[0].set_title(f'frequency = {frequencies[slice_f]:.0f} Hz')
        axs[0].set_xlabel('distance [cm]')

        axs[1].semilogy(frequencies, df_matrix[:, slice_d])
        axs[1].set_title(f'distance = {distances[slice_d]:.0f} cm')
        axs[1].set_xlabel('frequency [Hz]')
        plt.show()

    interactive_plot = interact(update_figure, 
                                slice_f=IntSlider(200, min=0, max=len(frequencies), continuous_update=False), 
                                slice_d=IntSlider(50, min=0, max=len(distances), continuous_update=False))

In [ ]:
def buzzer_signal(source):
    from signals import generate_signal
    kwargs = dict(
        duration_sec=30, 
        max_dB=-10,
        Fs=FS,
    )
    if source == 'mono4125':
        return generate_signal(signal_type='mono', frequency_hz=4125, **kwargs)
    elif source == 'random':
        return generate_signal(signal_type='random', **kwargs)
    elif source == 'multi':
        frequencies = get_frequencies(FS)
        phase = np.random.uniform(0, 2*np.pi)
        signal = generate_signal(signal_type='mono', frequency_hz=frequencies[1], **kwargs, phase_offset=phase)
        for f in frequencies[2:]:
            phase = np.random.uniform(0, 2*np.pi)
            signal += generate_signal(signal_type='mono', frequency_hz=f, **kwargs, phase_offset=phase)
        return signal

    
signals = {}
for key in ['mono4125', 'random', 'multi']:
    try:
        signal = pd.read_pickle(f'results/{key}.pk')
    except:
        signal = buzzer_signal(key)
        pd.to_pickle(signal, f'results/{key}.pk')
    signals[key] = signal

In [ ]:
fig, ax = plt.subplots()
room = generate_room(distance_cm=50, yaw_deg=0, ax=ax, single_mic=True)

In [ ]:
fig, axs = plt.subplots(1, 2)
fig.set_size_inches(15, 5)
room = generate_room(distance_cm=20, yaw_deg=0, ax=axs[0])
axs[0].set_title(0)
room = generate_room(distance_cm=50, yaw_deg=27, ax=axs[1])
axs[1].set_title(27)

In [ ]:
for distance_cm in [10, 50]:
    fig, axs = plt.subplots(1, 2)
    fig.suptitle(distance_cm)
    frequencies_theo = get_frequencies(FS)
    Hs = generate_rir(distance_cm=distance_cm, single_mic=True, ax=axs[0])
    for H in Hs:
        axs[1].plot(frequencies_theo, np.abs(H))

In [ ]:
#room = generate_room(distance_cm=50, single_mic=True, source='random')
room = generate_room(distance_cm=50, single_mic=True, source='multi')
signals_f = get_signals_f(room)
frequencies = get_frequencies(room.fs)

fig, axs = plt.subplots(signals_f.shape[1], 1, sharex=True, squeeze=False)
fig.set_size_inches(8, 2.5 * signals_f.shape[1])
for i in range(signals_f.shape[1]):
    axs[i, 0].pcolormesh(range(signals_f.shape[0]), frequencies, np.log10(np.abs(signals_f[:, i].T)))
    axs[i, 0].set_ylabel("frequency [Hz]")
    axs[i, 0].set_title(f"mic{i}")
axs[i, 0].set_xlabel("time index")

In [ ]:
room.compute_rir()

n_times = 500
times = np.arange(n_times) / FS
plt.plot(times, room.rir[0][0][:n_times])
plt.show()

# Distance-frequency matrix

In [ ]:
distances = np.linspace(0, 50, 100)
delta_f = 30
frequencies = get_frequencies(FS)[delta_f:-delta_f]
df_matrix = np.empty((len(frequencies), len(distances)))

for j, distance_cm in enumerate(distances):
    room = generate_room(distance_cm=distance_cm, single_mic=True, source='multi')
    
    signals_f = get_signals_f(room)
    spec = np.sum(np.abs(signals_f), axis=1) / (signals_f.shape[1] * N_BUFFER)
    spec_time_avg = np.mean(spec, axis=0)
    df_matrix[:, j] = spec_time_avg[delta_f:-delta_f]

In [ ]:
plt.figure()
plt.pcolormesh(distances, frequencies, np.log10(df_matrix))
matrix_slices(df_matrix, frequencies, distances)

In [ ]:
distances_theo = np.linspace(0, 50, 100)

frequencies_theo = get_frequencies(FS)[delta_f:-delta_f]
df_matrix_theo = np.empty((len(frequencies_theo), len(distances_theo)))

for j, distance_cm in enumerate(distances_theo):
    H = generate_rir(distance_cm=distance_cm, single_mic=True)[0]
    df_matrix_theo[:, j] = np.abs(H)

In [ ]:
plt.figure()
plt.pcolormesh(distances_theo, frequencies_theo, np.log10(df_matrix_theo))
matrix_slices(df_matrix_theo, frequencies_theo, distances_theo)

# Experiment twin

In [ ]:
df_total = pd.DataFrame(columns=['signals_f', 'degree', 'yaw', 'distance', 'source', 'psd', 'spec', 'frequencies'])

exp_name = '2020_11_23_wall2'; 
source = 'mono4125'
n_times = 300

frequencies = get_frequencies(FS)

for distance_cm in np.arange(100, step=10):
    for degree in [0, 27, 54, 81]:
        room = generate_room(distance_cm=distance_cm, yaw_deg=degree)

        N_BUFFER = 2**10
        signals_f = get_signals_f(room)[:n_times]

        yaw = np.full(signals_f.shape[0], -degree)

        spec = np.sum(np.abs(signals_f), axis=1)

        # TODO(FD): implement if needed. 
        psd = None 

        df_total.loc[len(df_total), :] = dict(
            degree=degree,
            yaw=yaw,
            distance=distance_cm,
            source=str(source),
            signals_f=signals_f,
            frequencies=frequencies,
            spec=spec,
            psd=psd
        ) 
        
fname = f'results/{exp_name}_simulated.pkl'
pd.to_pickle(df_total, fname)
print('saved as', fname)